In [45]:
import time
import pandas as pd
import requests

from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

- 내 위치에 있는 파일 가져올때

In [44]:
import os
import subprocess 

notepad = os.environ['WINDIR']+'\\system32\\notepad.exe'
driver_name =  os.environ['HOMEDRIVE'] + os.environ['HOMEPATH'] + '\\downloads\\chromedriver.exe'


- 네이버 지식iN : 어버이날 선물로

In [59]:
sword = '어버이날 선물'

# nurl = 'https://kin.naver.com/search/list.naver?query=' + sword
naver = 'https://www.naver.com/'
driver = webdriver.Chrome(driver_name)

In [60]:
# 네이버로 가기
driver.get(naver)
time.sleep(2)

In [61]:
# 지식iN 클릭
driver.find_element(By.XPATH, '//*[@id="NM_FAVORITE"]/div[1]/ul[1]/li[4]/a').click()

In [62]:
#popup 클릭
try:
    driver.find_element(By.XPATH, '//*[@id="eventPopupLayer2023SeasonFive"]/div[1]/div[1]/a').click()   
except:
    pass

In [66]:
# 검색어 넣기
driver.find_element(By.NAME, 'query').send_keys("어버이날 선물")

In [67]:
# 검색 버튼 클릭
driver.find_element(By.CLASS_NAME, 'search_btn').click()

- 전체 개수 구하기

In [68]:
int(driver.find_element(By.CLASS_NAME, 'number').text.strip('()').split('/')[-1].replace(',',''))

83856

In [69]:
urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
len(urls)

10

In [70]:
urls[0].text

'어버이날 선물 뭐가 좋을까요 2023.04.11.\n이번에 어버이날 선물 겸 부모님 생신 선물 하려고해요~ 요즘 친정엄마는 어깨가 아프시다시고, 아빠는 무릎때문에 병원 다니신다해요..ㅜㅜ 매번 현금으로만 드렸는데 이번엔...\n#어버이날선물\nQ&A > 자동차 수리,정비 | 답변수 2  추천수 17 | 답변 카놀라(qowh****)'

In [71]:
url_list = []
for url in urls:
    href = url.find_element(By.TAG_NAME, 'a')
    url_list.append(href.get_attribute('href'))    

- 2page로 이동

In [72]:
driver.find_element(By.XPATH, '//*[@id="s_content"]/div[3]/div[2]/a[1]').click()
time.sleep(2)

In [73]:
urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
len(urls)

10

In [74]:
for url in urls:
    href = url.find_element(By.TAG_NAME, 'a')
    url_list.append(href.get_attribute('href'))   

In [75]:
len(url_list)

20

In [76]:
driver.close()

- 반복해서 가져와 보기

In [77]:
def get_total_page(cnt):
    if cnt > 100:
        return 10
    else:
        return int(cnt / 10)


In [78]:

driver = webdriver.Chrome(driver_name)
driver.get(naver)
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="NM_FAVORITE"]/div[1]/ul[1]/li[4]/a').click()
try:
    driver.find_element(By.XPATH, '//*[@id="eventPopupLayer2023SeasonFive"]/div[1]/div[1]/a').click()   
except:
    pass

driver.find_element(By.NAME, 'query').send_keys("어버이날 선물")
driver.find_element(By.CLASS_NAME, 'search_btn').click()


urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')

# 전체 지식in 조회 수
cnt = int(driver.find_element(By.CLASS_NAME, 'number').text.strip('()').split('/')[-1].replace(',',''))
total_page = get_total_page(cnt)
print(total_page)

url_list = []
npage = 1
while True:    
    for url in urls:

        href = url.find_element(By.TAG_NAME, 'dl')
        # 답변수
        hit = int(href.find_element(By.CLASS_NAME, 'hit').text.split()[-1].strip())

        href = href.find_element(By.TAG_NAME, 'a')
        # 제목
        title = href.text
        # 답변 URL
        uurl = href.get_attribute('href')
        
        url_list.append({'제목':title, 'URL': uurl, '답변수':hit})

    if npage >= total_page:
        break

    driver.find_element(By.XPATH, f'//*[@id="s_content"]/div[3]/div[2]/a[{npage}]').click()
    time.sleep(2)

    urls = driver.find_elements(By.XPATH, '//*[@id="s_content"]/div[3]/ul/li')
    npage += 1

driver.close()

df = pd.DataFrame(url_list)
df.head()


10


,제목,URL,답변수
0,어버이날 선물 뭐가 좋을까요,https://kin.naver.com/qna/detail.naver?d1id=8&...,2
1,어버이날 선물 추천 부탁해요~,https://kin.naver.com/qna/detail.naver?d1id=3&...,2
2,어버이날 골프 관련 선물,https://kin.naver.com/qna/detail.naver?d1id=10...,7
3,어버이날에 꼭 선물을 드려야 되나요?,https://kin.naver.com/qna/detail.naver?d1id=8&...,6
4,40대 아버지 어버이날 선물,https://kin.naver.com/qna/detail.naver?d1id=8&...,20


In [79]:
print(len(url_list))

100


- sub 답변들 가져오기

In [122]:
driver = webdriver.Chrome(driver_name)
        
with open('data/어버이날선물.txt', 'w', encoding='utf-8') as f:

    for index in df.index:
        
        driver.get(df.URL[index])
        time.sleep(2)
        # print((df.답변수[index]//5))

        for i in range((df.답변수[index]//5)):
            
            try: 
                driver.find_element(By.ID, 'nextPageButton').click()
                time.sleep(2)
            except:
                continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        answer_list = soup.select('.answer-content__item._contentWrap._answer')

        for answer in answer_list:
            texts = answer.select('.se-module.se-module-text > p')
            
            for text in texts:
                f.write(text.get_text() + '\n')
    
    
driver.close()